In [ ]:
import numpy as np
import pickle

In [ ]:
class Particle:
    def __init__(self, x0, y0, vx0, vy0, ID):
        self.ID = ID
        self.x  = x0
        self.y  = y0
        self.vx = vx0
        self.vy = vy0



class Segment:
    def __init__(self, x1, y1, x2, y2, ID):
        self.ID = ID
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.collisions = []



class ParticleEngine:
    def __init__(self, collide_radius, dt, n_steps, xmin, xmax, ymin, ymax, grid_nx, grid_ny, pressure_grid_nx, pressure_grid_ny):
        self.particles = []
        self.segments  = []
        
        self.collide_radius = collide_radius

        self.delta_t = dt
        self.n_steps = n_steps

        self.particle_history = None
        self.pressure_history = None

        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax

        self.gridcells_x = grid_nx
        self.gridcells_y = grid_ny

        self.press_nx = pressure_grid_nx
        self.press_ny = pressure_grid_ny

        self.grid_xs = np.linspace(xmin, xmax, grid_nx+1)
        self.grid_ys = np.linspace(ymin, ymax, grid_ny+1)

        # Linear grid cells storage
        # ID = ix + iy*Nx
        self.grid_cells = [[] for i in range(grid_nx*grid_ny)    ]
        self.grid_cells_seg = [[] for i in range(grid_nx*grid_ny)]

    # Recalculates what particles are in corresponding grid cells
    def update_grid(self):
        self.grid_cells = [[] for i in range(self.gridcells_x*self.gridcells_y)    ]
        self.grid_cells_seg = [[] for i in range(self.gridcells_x*self.gridcells_y)]
        
        # 1. Calculating which particle is contained by which grid cell
        # part = Particle() instance
        for i, part in enumerate(self.particles):
            gridcell_ix = round((part.x - self.xmin)/(self.xmax - self.xmin)*self.gridcells_x)
            gridcell_iy = round((part.y - self.ymin)/(self.ymax - self.ymin)*self.gridcells_y)

            if (gridcell_ix < 0 or gridcell_ix >= self.gridcells_x or gridcell_iy < 0 or gridcell_iy >= self.gridcells_y):
                # Particle has left simulation area
                continue

            gridcell_id = gridcell_ix + gridcell_iy*self.gridcells_x
            self.grid_cells[gridcell_id].append(i)

        # 2. Calculating which segment is contained by which grid cell
        # seg = [x1, y1, x2, y2] list
        for i, seg in enumerate(self.segments):
            x1, y1, x2, y2 = tuple(seg)
            grid_cell_ix_1 = round((x1 - self.xmin)/(self.xmax - self.xmin)*self.gridcells_x)
            grid_cell_iy_1 = round((y1 - self.ymin)/(self.ymax - self.ymin)*self.gridcells_y)
            grid_cell_ix_2 = round((x2 - self.xmin)/(self.xmax - self.xmin)*self.gridcells_x)
            grid_cell_iy_2 = round((y2 - self.ymin)/(self.ymax - self.ymin)*self.gridcells_y)

            # No "leftance" check since segments are static

            gridcell_id_1 = grid_cell_ix_1 + grid_cell_iy_1*self.gridcells_x
            gridcell_id_2 = grid_cell_ix_2 + grid_cell_iy_2*self.gridcells_x

            self.grid_cells_seg[gridcell_id_1].append(i)
            if (gridcell_id_1 != gridcell_id_2):
                self.grid_cells_seg[gridcell_id_2].append(i)
            

    def update_pressure_grid(self):
        self.pressure_grid = np.zeros((self.press_nx, self.press_ny))
        for i, part in enumerate(self.particles):
            gridcell_ix = round((part.x - self.xmin)/(self.xmax - self.xmin)*self.press_nx)
            gridcell_iy = round((part.y - self.ymin)/(self.ymax - self.ymin)*self.press_ny)

            if (gridcell_ix < 0 or gridcell_ix >= self.press_nx or gridcell_iy < 0 or gridcell_iy >= self.press_ny):
                # Particle has left simulation area
                continue

            self.pressure_grid[gridcell_ix, gridcell_iy] += 1

    def particle_dist(self, p1, p2):
        return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

    def init_hist(self):
        self.particle_history = [[] for i in range(len(self.particles))]
        self.pressure_history = []

    def save_hist(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.particle_history, f)
        with open('Pressure.pkl', 'wb') as f:
            pickle.dump(self.pressure_history, f)

    def save_config(self):
        config = {
            'xmin': self.xmin,
            'xmax': self.xmax,
            'ymin': self.ymin, 
            'ymax': self.ymax,
            'segments': self.segments
        }
        with open('Config.pkl', 'wb') as f:
            pickle.dump(config, f)

    # Saves all particles state at current time step
    def save_frame(self):
        for i, part in enumerate(self.particles):
            self.particle_history[i].append((part.x, part.y))

    def add_particle(self, particle):
        self.particles.append(particle)
    
    def add_segment(self, segment):
        self.segments.append(segment)

    # Checks whether two particles are close enough to consider it as collision
    def two_particles_collide(self, p1, p2):
        dist = self.particle_dist(p1, p2)
        if (dist <= 2*self.collide_radius):
            return True
        return False

    # If two particles collide. recalculates their velocity vectors
    def recalc_particles_vect(self, p1, p2):

        m_A = m_B = 1.0

        v_A = np.array([p1.vx, p1.vy])
        v_B = np.array([p2.vx, p2.vy])
        r_A = np.array([p1.x,  p1.y] )
        r_B = np.array([p2.x,  p2.y] )

        # Relative position and relative velocity vectors
        r_rel = r_B - r_A
        v_rel = v_B - v_A

        # Distance between particles
        distance = np.linalg.norm(r_rel)

        # Direction of impact (unit vector along the line of impact)
        impact_dir = r_rel / distance

        # Project velocities along the impact direction
        v_A_proj = np.dot(v_A, impact_dir)
        v_B_proj = np.dot(v_B, impact_dir)

        # Calculate new velocities along the line of impact
        v_A_new_proj = ((m_A - m_B) * v_A_proj + 2 * m_B * v_B_proj) / (m_A + m_B)
        v_B_new_proj = ((m_B - m_A) * v_B_proj + 2 * m_A * v_A_proj) / (m_A + m_B)

        # Update the velocity vectors
        v_A_new = v_A + (v_A_new_proj - v_A_proj) * impact_dir
        v_B_new = v_B + (v_B_new_proj - v_B_proj) * impact_dir

        return v_A_new, v_B_new

    # Recalculating velocities of all particles based on collisions
    def recalc_velocities(self):
        for ix in range(self.gridcells_x):
            for iy in range(self.gridcells_y):
                gridcell_id = ix + iy*self.gridcells_x

                self.recalc_gridcell(gridcell_id)


    def particle_and_segment_collide(self, p, seg):
        """
        Calculates whether a particle will collide with a segment and recalculates its velocity in case of collision.

        Args:
        x, y: Initial coordinates of the particle.
        vx, vy: Velocity vector of the particle.
        x1, y1, x2, y2: Coordinates of the ends of the segment.
        threshold: Distance threshold for collision.

        Returns:
        collision: Boolean indicating whether a collision occurs.
        new_velocity: New velocity vector of the particle after collision, if collision occurs.
        """
        threshold = self.collide_radius
        x, y = p.x, p.y
        vx, vy = p.vx, p.vy
        x1, y1 = seg[0], seg[1]
        x2, y2 = seg[2], seg[3]

        # Convert all inputs to numpy arrays for vectorized operations
        p0 = np.array([x, y])
        v = np.array([vx, vy])
        s1 = np.array([x1, y1])
        s2 = np.array([x2, y2])

        # Function to calculate line intersection
        def line_intersection(p0, v, s1, s2):
            # Line equation for line 1 (particle trajectory): p = p0 + t * v
            # Line equation for line 2 (segment): q = s1 + u * (s2 - s1)
            # We need to solve for t and u such that p = q
            s = s2 - s1
            matrix = np.array([-v, s]).T
            if np.linalg.det(matrix) == 0:  # Lines are parallel
                return None
            t, u = -np.linalg.solve(matrix, s1 - p0)
            return t, u

        # Check if the particle trajectory intersects with the segment
        result = line_intersection(p0, v, s1, s2)
        if result is not None:
            t, u = result
            # Check if intersection point is within both the line segment and the particle's trajectory
            if 0 <= u <= 1 and t >= 0:
                #print(f'Intersection')
                intersection_point = p0 + t * v
                # Check if the intersection is within the threshold distance
                if np.linalg.norm(p0 - intersection_point) <= threshold:
                    # Collision detected, calculate new velocity
                    # For simplicity, assuming a perfect elastic collision with a straight line
                    # Find the normal vector to the segment
                    normal = np.array([s2[1] - s1[1], s1[0] - s2[0]])
                    normal = normal / np.linalg.norm(normal)  # Normalize
                    # Reflect the velocity vector across the normal
                    new_velocity = v - 2 * np.dot(v, normal) * normal
                    return True, new_velocity

        # No collision
        return False, None

    def recalc_gridcell(self, grid_cell_id):
        local_particles = self.grid_cells[grid_cell_id    ]
        local_segments  = self.grid_cells_seg[grid_cell_id]
        processed = []

        # Processing segment collision
        for i in local_particles:
            for j in local_segments:
                collide, new_v = self.particle_and_segment_collide(self.particles[i], self.segments[j]) # ToDo
                if (not collide):
                    continue
                processed.append(i)

                vx_new, vy_new = new_v[0], new_v[1]
                self.particles[i].vx = vx_new
                self.particles[i].vy = vy_new

        # Processing particle collision
        for i in local_particles:
            processed.append(i)
            for j in local_particles:
                if (j in processed):
                    continue
                collide = self.two_particles_collide(self.particles[i], self.particles[j])
                if (not collide):
                    continue
                processed.append(j)
                v1_new, v2_new = self.recalc_particles_vect(self.particles[i], self.particles[j])
                vx1_new, vy1_new = tuple(v1_new)
                vx2_new, vy2_new = tuple(v2_new)
                self.particles[i].vx = vx1_new
                self.particles[i].vy = vy1_new
                self.particles[j].vx = vx2_new
                self.particles[j].vy = vy2_new


    # Performs time step for all particles after velocity recalculations
    def step(self):
        for i in range(len(self.particles)):
            self.particles[i].x += self.particles[i].vx * self.delta_t
            self.particles[i].y += self.particles[i].vy * self.delta_t

    def simulation(self, update_step):
        self.init_hist()
        for i in range(self.n_steps):
            if (i%10 == 0):
                print(f'step {i}')

            self.save_frame()
            self.recalc_velocities()
            self.step()

            if (i % update_step == 0):
                self.update_grid()
                self.update_pressure_grid()
                self.pressure_history.append(self.pressure_grid)


In [ ]:
dt = 0.02
collision_radius = 1.8
n_steps = 2400

xmin = 25
xmax = 480
ymin = 25
ymax = 480

n_pressure = 40
n_collision = 40

engine = ParticleEngine(collision_radius, dt, n_steps, 25, 480, 25, 480, n_collision, n_collision, n_pressure, n_pressure)

N = 50
for i in range(2*N):
    for j in range(int(N*1.5)):
        particle = Particle(30 + i*4, -150 + j*4, 0, 15, 0)
        engine.add_particle(particle)

segments = [
    [220, 200, 240, 220], # 1
    [220, 200, 200, 220], # 2
    [240, 220, 250, 240], # 3
    [200, 220, 190, 240], # 4
    [250, 240, 250, 460], # 5
    [190, 240, 190, 460]
]
for s in segments:
    engine.add_segment(s)

In [ ]:
engine.update_grid()

In [ ]:
lens = [len(gridcell) for gridcell in engine.grid_cells]
print(f'BottleNeck: max len = {max(lens)}')
print(f'Average num particles in grid cell: {np.mean(np.array(lens))}')

In [ ]:
engine.simulation(5)

In [ ]:
engine.save_hist(filename = 'Hist1.pkl')
engine.save_config()

In [ ]:
with open('Pressure.pkl', 'wb') as f:
    pickle.dump(engine.pressure_history, f)